# 使用聊天模型和提示模板构建一个简单的LLM应用

在这个快速入门中，我们将向您展示如何使用LangChain构建一个简单的LLM应用。该应用将把文本从英文翻译成另一种语言。这是一个相对简单的LLM应用——它仅包含一次LLM调用和一些提示。尽管如此，这对于开始使用LangChain来说是一个绝佳的方式——许多功能只需一些提示和一次LLM调用就可以实现！

阅读完本教程后，您将对以下内容有高层次的了解：

- 使用[语言模型](/docs/concepts/chat_models)

- 使用[提示模板](/docs/concepts/prompt_templates)

- 使用[LangSmith](https://docs.smith.langchain.com/)调试和追踪您的应用

让我们开始吧！

## 设置

### 安装

要安装LangChain，请运行以下命令：

```{=mdx}
import Npm2Yarn from '@theme/Npm2Yarn';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```

有关更多细节，请参阅我们的[安装指南](/docs/how_to/installation/)。

### LangSmith

您使用LangChain构建的许多应用将包含多个步骤和多次LLM调用。
随着这些应用变得越来越复杂，能够检查您的链或代理内部发生了什么变得至关重要。
实现此目的的最佳方式是使用[LangSmith](https://smith.langchain.com)。

在上方链接注册后，请确保设置您的环境变量以开始记录追踪信息：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."

# 如果您不在无服务器环境中，可减少追踪延迟
# export LANGCHAIN_CALLBACKS_BACKGROUND=true
```

## 使用语言模型

首先，我们来学习如何单独使用语言模型。LangChain支持许多不同的语言模型，您可以互换使用它们。有关使用特定模型的详细信息，请参阅[支持的集成](/docs/integrations/chat/)。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`{ model: "gpt-4" }`} />
```

In [2]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

首先，我们直接使用模型。[聊天模型](/docs/concepts/chat_models)是LangChain [Runnables](/docs/concepts/runnables/)的实例，这意味着它们提供了一个标准接口用于与之交互。要简单调用模型，我们可以将一组[消息](/docs/concepts/messages/)传递给`.invoke`方法。

In [3]:
import { HumanMessage, SystemMessage } from "@langchain/core/messages"

const messages = [
  new SystemMessage("将以下内容从英文翻译成意大利语"),
  new HumanMessage("hi!"),
];

await model.invoke(messages)

AIMessage {
  "id": "chatcmpl-AekSfJkg3QIOsk42BH6Qom4Gt159j",
  "content": "Ciao!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 20,
      "completionTokens": 3,
      "totalTokens": 23
    },
    "finish_reason": "stop",
    "usage": {
      "prompt_tokens": 20,
      "completion_tokens": 3,
      "total_tokens": 23,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_6fc10e10eb"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 3,
    "input_tokens": 20,
    "total_tokens": 23,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
    "output_token_details": {
      "audio": 0,
      "reasoning": 0
    }
  }


```{=mdx}
:::tip

如果我们启用了LangSmith，我们可以看到此运行记录已发送到LangSmith，并可以查看[LangSmith追踪](https://smith.langchain.com/public/45f1a650-38fb-41e1-9b61-becc0684f2ce/r)。LangSmith追踪报告了[令牌](/docs/concepts/tokens/)使用情况、延迟、[标准模型参数](/docs/concepts/chat_models/#standard-parameters)（如温度）以及其他信息。

:::
```

请注意，聊天模型接收[消息](/docs/concepts/messages/)对象作为输入并生成消息对象作为输出。除了文本内容外，消息对象还传达对话[角色](/docs/concepts/messages/#role)，并包含重要数据，如[工具调用](/docs/concepts/tool_calling/)和令牌使用计数。

LangChain还支持通过字符串或[OpenAI格式](/docs/concepts/messages/#openai-format)进行聊天模型输入。以下内容是等效的：

```javascript
await model.invoke("Hello")

await model.invoke([ {role: "user", content: "Hello" }])

await model.invoke([new HumanMessage("hi!")])
```

### 流式传输

由于聊天模型是[Runnables](/docs/concepts/runnables/)，它们提供了一个标准接口，其中包括异步和流式调用模式。这允许我们从聊天模型中逐个流式传输令牌：

In [5]:
const stream = await model.stream(messages);

const chunks = [];
for await (const chunk of stream) {
  chunks.push(chunk);
  console.log(`${chunk.content}|`);
}

|
C|
iao|
!|
|
|


## 提示模板

目前我们是直接将一组消息列表传递给语言模型。这一组消息来自哪里呢？通常，它是由用户输入和应用逻辑组合构建的。这种应用逻辑通常将原始用户输入转换为准备好传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。

[提示模板](/docs/concepts/prompt_templates/)是LangChain中的一个概念，旨在帮助完成这种转换。它们接收原始用户输入并返回准备好传递给语言模型的数据（提示）。

让我们在此创建一个提示模板。它将接收两个用户变量：

- `language`：要将文本翻译成的语言
- `text`：要翻译的文本

In [6]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

首先，让我们创建一个字符串，该字符串将用于格式化系统消息：

In [7]:
const systemTemplate = "将以下内容从英文翻译成 {language}"

接下来，我们可以创建提示模板。这将是`systemTemplate`以及用于放置文本的更简单模板的组合

In [8]:
const promptTemplate = ChatPromptTemplate.fromMessages(
  [
    ["system", systemTemplate],
    ["user", "{text}"]
  ]
)

请注意，`ChatPromptTemplate`支持单个模板中的多个[消息角色](/docs/concepts/messages/#role)。我们将`language`参数格式化到系统消息中，将用户`text`格式化到用户消息中。

此提示模板的输入是一个字典。我们可以单独使用这个提示模板来查看它的效果

In [9]:
const promptValue = await promptTemplate.invoke({ language: "意大利语", text: "hi!" })

promptValue

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      SystemMessage {
        "content": "将以下内容从英文翻译成 意大利语",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      HumanMessage {
        "content": "hi!",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ]
  },
  lc_namespace: [ 'langchain_core', 'prompt_values' ],
  messages: [
    SystemMessage {
      "content": "将以下内容从英文翻译成 意大利语",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    HumanMessage {
      "content": "hi!",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  ]
}


我们可以看到它返回了一个包含两条消息的`ChatPromptValue`。如果我们想直接访问这些消息，可以这样做：

In [10]:
promptValue.toChatMessages()

[
  SystemMessage {
    "content": "将以下内容从英文翻译成 意大利语",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "hi!",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]


最后，我们可以在格式化后的提示上调用聊天模型：

In [11]:
const response = await model.invoke(promptValue)
console.log(`${response.content}`)

Ciao!


如果我们查看LangSmith追踪，可以看到所有三个组件都显示在[LangSmith追踪](https://smith.langchain.com/public/6529d912-8564-4686-8df8-999c427621a7/r)中。

## 结论

就是这样！在本教程中，您已经学习了如何创建您的第一个简单的LLM应用。您已经学习了如何使用语言模型、如何创建提示模板，以及如何通过LangSmith获得对您创建的应用的可观测性。

这仅是成为一名熟练的AI工程师所需知识的冰山一角。幸运的是——我们还有许多其他资源！

有关LangChain核心概念的进一步阅读，我们提供了详细的[概念指南](/docs/concepts)。

如果您对这些概念有更具体的问题，请查看以下如何指南部分：

- [聊天模型](/docs/how_to/#chat-models)
- [提示模板](/docs/how_to/#prompt-templates)

以及LangSmith文档：

- [LangSmith](https://docs.smith.langchain.com)